In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
# training: original data
# training_tofill: transformed data
training=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Lending Club Project_Joint/training.csv')
training_tofill=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Lending Club Project_Joint/training_transformed.csv')

In [0]:
training.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,results,issue_y,issue_m
0,1294383,12000,12000,12000.0,36 months,7.89,375.43,A,A5,Nurse,2 years,RENT,75000.0,Source Verified,Feb-2015,n,NaN,NaN,debt_consolidation,Debt consolidation,902xx,CA,20.42,0.0,May-2009,0.0,NaN,NaN,6.0,0.0,685,9.5,16.0,w,0.0,0.0,12623.500000,12623.50,12000.00,623.50,...,48690.0,6000.0,56377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Fully Paid,2015,Feb
1,1876502,5000,5000,5000.0,36 months,6.99,154.37,A,A3,equipment operator,8 years,RENT,25000.0,Not Verified,Nov-2014,n,NaN,NaN,debt_consolidation,Debt consolidation,977xx,OR,7.97,0.0,Jul-2006,0.0,NaN,NaN,3.0,0.0,6162,63.5,6.0,f,0.0,0.0,5544.621617,5544.62,5000.00,544.62,...,6162.0,7700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Fully Paid,2014,Nov
2,954710,4000,4000,4000.0,36 months,9.17,127.52,B,B2,Collector / Biller,6 years,RENT,32000.0,Source Verified,Dec-2015,n,NaN,NaN,credit_card,Credit card refinancing,750xx,TX,11.52,0.0,Sep-2004,0.0,35.0,NaN,10.0,0.0,6312,69.4,18.0,f,0.0,0.0,4588.439074,4588.44,4000.00,588.44,...,6312.0,3700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Fully Paid,2015,Dec
3,809314,10000,10000,10000.0,36 months,6.99,308.73,A,A3,BUSINESS FINANCIAL MANAGER,10+ years,MORTGAGE,135000.0,Source Verified,Jan-2016,n,NaN,NaN,debt_consolidation,Debt consolidation,225xx,VA,23.66,0.0,May-1991,0.0,26.0,NaN,18.0,0.0,14802,58.0,35.0,w,0.0,0.0,9406.500000,9406.50,7107.03,1100.03,...,190254.0,3000.0,180574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,Y,Jan-2019,COMPLETE,Jul-2018,1384.0,45.0,6.0,Charged Off,2016,Jan
4,1069627,14000,14000,14000.0,36 months,7.89,438.00,A,A5,Managing Member,7 years,RENT,35000.0,Verified,Oct-2015,n,NaN,NaN,credit_card,Credit card refinancing,602xx,IL,10.22,0.0,Aug-1978,0.0,70.0,NaN,3.0,0.0,13385,63.4,16.0,w,0.0,0.0,15798.640798,15798.64,14000.00,1798.64,...,14021.0,21100.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Fully Paid,2015,Oct


In [0]:
training_tofill.head()

,id,acc_open_past_24mths,addr_state,application_type,avg_cur_bal,bc_open_to_buy,bc_util,desc,dti,dti_joint,home_ownership,inq_last_6mths,loan_amnt,num_bc_tl,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mths_since_recent_bc,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_il_tl,num_op_rev_tl,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,percent_bc_gt_75,purpose,revol_util,sec_app_revol_util,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,pct_tl_nvr_dlq,results,issue_d,issue_y,issue_m,earliest_cr_line1,sec_app_earliest_cr_line1,earliest_cr_line_min,verification_status_all,if_worse_rating,annual_inc_all,revol_bal_all,mort_acc_all,num_rev_accts_all,open_acc_all,if_revol_delinq,if_cur_il,if_delinq_2yrs,if_pub_rec,if_pub_rec_bankruptcies,if_chargeoff_12_mths,if_collections_12_mths
0,1294383,5.0,CA,Individual,8115.0,5315.0,11.4,0,20.42,NaN,RENT,0.0,12000,1.0,58.0,69.0,22.0,69.0,1.0,1.0,1.0,12.0,2.0,1.0,6.0,2.0,0.0,debt_consolidation,9.5,NaN,36 months,0.0,48690.0,68842.0,16.0,48690.0,100.0,Fully Paid,Feb-2015,2015,Feb,2009.0,NaN,2009.0,1.0,0.0,75000.0,685.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1876502,0.0,OR,Individual,2054.0,2069.0,73.1,0,7.97,NaN,RENT,0.0,5000,4.0,NaN,100.0,50.0,50.0,2.0,3.0,2.0,0.0,3.0,3.0,3.0,0.0,50.0,debt_consolidation,63.5,NaN,36 months,0.0,6162.0,9700.0,6.0,6162.0,100.0,Fully Paid,Nov-2014,2014,Nov,2006.0,NaN,2006.0,0.0,0.0,25000.0,6162.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,954710,0.0,TX,Individual,631.0,526.0,85.8,0,11.52,NaN,RENT,0.0,4000,10.0,135.0,129.0,41.0,96.0,6.0,9.0,6.0,3.0,10.0,9.0,10.0,0.0,83.3,credit_card,69.4,NaN,36 months,70.0,6312.0,9100.0,18.0,6312.0,88.9,Fully Paid,Dec-2015,2015,Dec,2004.0,NaN,2004.0,1.0,1.0,32000.0,6312.0,0.0,15.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,809314,7.0,VA,Individual,29161.0,3000.0,87.0,0,23.66,NaN,MORTGAGE,0.0,10000,5.0,124.0,296.0,7.0,12.0,1.0,12.0,5.0,11.0,12.0,10.0,18.0,4.0,100.0,debt_consolidation,58.0,NaN,36 months,0.0,524910.0,573138.0,35.0,190254.0,83.0,Charged Off,Jan-2016,2016,Jan,1991.0,NaN,1991.0,1.0,1.0,135000.0,14802.0,5.0,19.0,18.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1069627,0.0,IL,Individual,4674.0,7715.0,63.4,0,10.22,NaN,RENT,0.0,14000,7.0,161.0,445.0,61.0,61.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,0.0,50.0,credit_card,63.4,NaN,36 months,0.0,14021.0,22628.0,16.0,14021.0,87.5,Fully Paid,Oct-2015,2015,Oct,1978.0,NaN,1978.0,1.0,1.0,35000.0,13385.0,3.0,9.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
print('Original: ',training.shape)
print('To Replace: ',training_tofill.shape)

Original:  (1045085, 146)
To Replace:  (1045085, 58)


#1. Missing Value Analysis
After we transformed the raw data from Lending Club, there are still 36 columns contain null values. This may be caused by documentation methods changed and such.

The analysis is based on the original data, as it contains more complete information, whcih helps us understand the meaning of null values.

This section tried to understand the meaning behind each null value, and next section will provide alternatives replacing null values.

In [0]:
# Columns with missing value
print(training_tofill.columns[training_tofill.isnull().any()],'\n','Columns with Missing Value: ',len(training_tofill.columns[training_tofill.isnull().any()]))

Index(['acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'dti', 'dti_joint', 'inq_last_6mths', 'num_bc_tl', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mths_since_recent_bc',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_tl_bal_gt_0', 'num_sats',
       'num_tl_op_past_12m', 'percent_bc_gt_75', 'revol_util',
       'sec_app_revol_util', 'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim',
       'total_acc', 'total_bal_ex_mort', 'pct_tl_nvr_dlq', 'earliest_cr_line1',
       'sec_app_earliest_cr_line1', 'earliest_cr_line_min', 'annual_inc_all',
       'mort_acc_all', 'num_rev_accts_all', 'open_acc_all'],
      dtype='object') 
 Columns with Missing Value:  36


##1.1 acc_open_past_24mths
**Solution:** Set Nulls to 0

For null value will be converted as 0 since before 2012 Mar, no account open in past 24 months was marked as null, but after 2012 Mar, this value was marked 0 in this column.

In [0]:
pd.DataFrame(data=(training.loc[training.acc_open_past_24mths.isnull(),'issue_y'].value_counts(),
                   training.loc[training.acc_open_past_24mths==0,'issue_y'].value_counts().sort_index()),index=['null_count','zero_count']).T

,null_count,zero_count
2007,479.0,NaN
2008,1906.0,NaN
2009,4226.0,NaN
2010,10071.0,NaN
2011,17379.0,NaN
2012,5989.0,1949.0
2013,NaN,5973.0
2014,NaN,7136.0
2015,NaN,12394.0
2016,NaN,6993.0


##1.2 avg_cur_bal & tot_cur_bal
**Solution:** Set Nulls to 0

The null values in both avg_cur_bal and tot_cur_bal columns will be conveted to 0 since before 2012 Mar., thses two values were set to null in order to represent 0 balance. 

Additionally, there is some borrowers have no open account; thus, the avg_cur_bal were na since 0 can't be devided.

In [0]:
avg_cur_bal_na=training.loc[training.avg_cur_bal.isnull(),['id','issue_y','open_acc','avg_cur_bal','tot_cur_bal']]
tot_cur_bal_na=training.loc[training.tot_cur_bal.isnull(),['id','issue_y','open_acc','avg_cur_bal','tot_cur_bal']]
avg_cur_bal_zeros=training.loc[training.avg_cur_bal==0,['id','issue_y','open_acc','avg_cur_bal','tot_cur_bal']]
tot_cur_bal_zeros=training.loc[training.avg_cur_bal==0,['id','issue_y','open_acc','avg_cur_bal','tot_cur_bal']]
print('Count of Null value in avg_cur_bal: ',len(avg_cur_bal_na))
print('Count of Null value in tot_cur_bal:',len(tot_cur_bal_na))
print('Count of 0 in avg_cur_bal:',len(avg_cur_bal_zeros))
print('Count of 0 value in tot_cur_bal:',len(tot_cur_bal_zeros))

Count of Null value in avg_cur_bal:  56219
Count of Null value in tot_cur_bal: 56201
Count of 0 in avg_cur_bal: 274
Count of 0 value in tot_cur_bal: 274


In [0]:
print('Count of null value each year')
pd.DataFrame(data=[avg_cur_bal_na.issue_y.value_counts().sort_index(),
                   tot_cur_bal_na.issue_y.value_counts().sort_index()],index=['avg','tot'])

Count of null value each year


,2007,2008,2009,2010,2011,2012,2013,2014,2017,2018
avg,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,5.0,4.0,5.0,4.0
tot,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN


In [0]:
print('Count of 0 value each year')
pd.DataFrame(data=[avg_cur_bal_zeros.issue_y.value_counts().sort_index(),
                   tot_cur_bal_zeros.issue_y.value_counts().sort_index()],index=['avg','tot'])

Count of 0 value each year


,2012,2013,2014,2015,2016,2017,2018
avg,7,14,32,34,33,59,95
tot,7,14,32,34,33,59,95


In [0]:
# Null value in avg_cur_bal that caused by no open account 
avg_cur_bal_na.loc[avg_cur_bal_na.open_acc==0,'issue_y'].value_counts().sort_index()

2013    3
2014    3
2017    3
2018    3
Name: issue_y, dtype: int64

In [0]:
avg_cur_bal_na.loc[avg_cur_bal_na.issue_y.isin(avg_cur_bal_na.loc[avg_cur_bal_na.open_acc==0,'issue_y'].value_counts().index)]

,id,issue_y,open_acc,avg_cur_bal,tot_cur_bal
216118,1781574,2013,0.0,NaN,0.0
225513,385152,2018,0.0,NaN,0.0
265537,2213988,2017,0.0,NaN,0.0
265566,2061754,2014,0.0,NaN,0.0
349583,1749083,2013,2.0,NaN,0.0
362107,1736683,2013,0.0,NaN,0.0
430750,2029154,2014,1.0,NaN,0.0
445069,1803189,2013,0.0,NaN,0.0
453099,2059431,2014,0.0,NaN,0.0
473306,2173926,2017,0.0,NaN,0.0


##1.3 dti & dti_join

**Solution:**
- dti: All null value in dti having a joint application, so in this case, use dti_joint instead.
- dti_joint: All the null come from individual applicaiton, except case (ID: 992525), in this special case, use the dti instead.

For other joint applications, use the weighted average of dti and dti_joint. The final dti can be calculated as annul_inc/(annual_inc + annual_inc_joint) * dti + annul_inc_joint/(annual_inc + annual_inc_joint) * dti_joint


**Note:** Co-borrowers start on 2015, so before that any second borrower related columns having NA before 2015 should carefully handle case by case.

In [0]:
dti_na=training.loc[training.dti.isnull(),['id','issue_y','emp_length','annual_inc','annual_inc_joint','application_type','dti_joint']]
print('Count of null dti: ',len(dti_na))

Count of null dti:  239


In [0]:
# This show counts of null dti in each year
# Most of the null dti applicants also don't have employment
dti_na.groupby('issue_y').count()

,id,emp_length,annual_inc,annual_inc_joint,application_type,dti_joint
issue_y,,,,,,
2016,28,1,28,28,28,28
2017,129,9,129,129,129,129
2018,82,1,82,82,82,82


In [0]:
# Most null value in dti also don't have income
dti_na.annual_inc.value_counts()

0.0    230
1.0      8
2.0      1
Name: annual_inc, dtype: int64

In [0]:
# In the whole dataset, all no income cases don't have dti
# as dti=debt/income, and income can't be zero in this case
training.loc[training.annual_inc==0,'id'].count()

230

In [0]:
# All null dti applicants have a co-borrower
print('Application Type in Null dti loans: ',dti_na.application_type.value_counts())
print('Number of Null dti in coborrower: ',training.dti_joint.isna().sum())

Application Type in Null dti loans:  Joint App    239
Name: application_type, dtype: int64
Number of Null dti in coborrower:  1026576


In [0]:
# Null dti_joint counts matches Individual applicants
training.application_type.value_counts(dropna=False)

Individual    1026576
Joint App       18509
Name: application_type, dtype: int64

##1.4 bc_open_to_buy
**Solution:** Set all nulls to 0

Definition: The difference between the credit limit assigned to a cardholder account and the present balance on the account. (on revolving account)

In [0]:
bc_open_to_buyna=training.loc[training.bc_open_to_buy.isnull(),['id','issue_y','num_rev_accts','revol_bal','revol_util','num_actv_rev_tl']]
bc_open_to_buyzeros=training.loc[training.bc_open_to_buy==0,['id','issue_y','num_rev_accts','revol_bal','revol_util','num_actv_rev_tl']]

In [0]:
pd.DataFrame(data=(bc_open_to_buyna.issue_y.value_counts().sort_index(),bc_open_to_buyzeros.issue_y.value_counts().sort_index()),index=['Null_Count','Zero_Count']).T

,Null_Count,Zero_Count
2007,479.0,NaN
2008,1906.0,NaN
2009,4226.0,NaN
2010,10071.0,NaN
2011,17379.0,NaN
2012,6421.0,779.0
2013,798.0,2111.0
2014,1863.0,3573.0
2015,2898.0,5872.0
2016,2383.0,3016.0


In [0]:
bc_open_to_buyna.revol_bal.value_counts()

0        2023
298        14
255        14
1112       12
10         12
         ... 
9098        1
15241       1
19335       1
23429       1
30693       1
Name: revol_bal, Length: 23754, dtype: int64

In [0]:
bc_open_to_buyna.revol_util.value_counts()

0.00     1550
66.70      74
54.10      73
55.80      72
63.00      72
         ... 
10.61       1
17.78       1
7.28        1
43.38       1
58.19       1
Name: revol_util, Length: 1154, dtype: int64

##1.5 mort_acc & sec_app_mort_acc
**Solution:** Set all nulls to 0

- mort_acc: null and 0 are interchangable, as before 2012, it's marked as null but after it's marked as 0

- sec_app_mort_acc: null and 0 are interchangable, as before 2017, it's marked as null but after it's marked as 0

Note: Co-borrower program started in 2015

In [0]:
# mort_acc
# Null value disappear in 2012, and zero value starts appear in 2012
mort_acc_na=training.loc[training.mort_acc.isnull(),['id','issue_y','mort_acc']]
mort_acc_zeros=training.loc[training.mort_acc==0,['id','issue_y','mort_acc']]
pd.DataFrame(data=(mort_acc_na.issue_y.value_counts().sort_index(),mort_acc_zeros.issue_y.value_counts().sort_index()),index=['Null_Count','Zero_Count']).T

,Null_Count,Zero_Count
2007,479.0,NaN
2008,1906.0,NaN
2009,4226.0,NaN
2010,10071.0,NaN
2011,17379.0,NaN
2012,5989.0,17048.0
2013,NaN,41221.0
2014,NaN,68938.0
2015,NaN,121952.0
2016,NaN,88098.0


In [0]:
# sec_app_mort_acc
# Select Null value and zero value in joint application loans
# Null value disappear in 2017, and zero value starts appear in 2017
sec_app_mort_acc_na=training.loc[(training.application_type=='Joint App') & (training.sec_app_mort_acc.isnull()),['id','issue_y','mort_acc']]
sec_app_mort_acc_zeros=training.loc[(training.application_type=='Joint App') & (training.sec_app_mort_acc==0),['id','issue_y','mort_acc']]
pd.DataFrame(data=(sec_app_mort_acc_na.issue_y.value_counts().sort_index(),sec_app_mort_acc_zeros.issue_y.value_counts().sort_index()),index=['Null_Count','Zero_Count']).T

,Null_Count,Zero_Count
2015,306.0,NaN
2016,3917.0,NaN
2017,1117.0,3106.0
2018,NaN,1629.0


##1.6 mths_since_recent_bc & num_bc_tl
**Solution:**
- mths_since_recent_bc: Set null to max(mths_since_recent_bc)+1=640
- num_bc_tl: set null to 0

**mths_since_recent_bc:** 0 means the most recent opened bank accounr after 2012. Before 2012, null means no bank account, but after 2012, the null has multiple interpretation, thus, we remain it as max(mths_since_recnet_bc)=639 for now.

**num_bc_tl:** before 2012 was null means no bank account, after it was replaced with 0

In [0]:
mths_since_recent_bc_na=training.loc[training.mths_since_recent_bc.isnull(),['issue_y','id','num_bc_tl']]
mths_since_recent_bc_zeros=training.loc[training.mths_since_recent_bc==0,['issue_y','id','num_bc_tl']]
num_bc_tl_na=training.loc[training.num_bc_tl.isnull(),['issue_y']]
num_bc_tl_zeros=training.loc[training.num_bc_tl==0,['issue_y']]

In [0]:
pd.DataFrame(data=(mths_since_recent_bc_na.issue_y.value_counts().sort_index(),
                   mths_since_recent_bc_zeros.issue_y.value_counts().sort_index(),
                   num_bc_tl_na.issue_y.value_counts().sort_index(),
                   num_bc_tl_zeros.issue_y.value_counts().sort_index()),
             index=['mths_na','mths_zeros','num_bc_na','num_bc_zeros']).T

,mths_na,mths_zeros,num_bc_na,num_bc_zeros
2007,479.0,NaN,479.0,NaN
2008,1906.0,NaN,1906.0,NaN
2009,4226.0,NaN,4226.0,NaN
2010,10071.0,NaN,10071.0,NaN
2011,17379.0,NaN,17379.0,NaN
2012,6367.0,248.0,22140.0,28.0
2013,699.0,754.0,NaN,83.0
2014,1718.0,1185.0,NaN,313.0
2015,2783.0,1614.0,NaN,637.0
2016,2231.0,1344.0,NaN,563.0


In [0]:
training.mths_since_recent_bc.max()

639.0

## 1.7 num_actv_bc_tl & num_actv_rev_tl & num_bc_sats & num_il_tl & num_op_rev_tl & num_rev_accts & sec_app_num_rev_accts & num_sats & num_tl_op_past_12m & tot_coll_amt & tot_cur_bal & total_bal_ex_mort & bc_util & num_bc_tl & mo_sin_old_il_acct & mo_sin_old_rev_tl_op & mo_sin_rcnt_rev_tl_op

**Solution:**
1. Set null to 0
2. Drop columns: bc_util & mo_sin_old_il_acct & mo_sin_old_rev_tl_op

Kept columns have null and 0 interchangable

In [0]:
# num_actv_bc_tl
num_actv_bc_na=training.loc[training.num_actv_bc_tl.isnull(),['issue_y','id']]
num_actv_bc_zeros=training.loc[training.num_actv_bc_tl==0,['issue_y','id']]
pd.DataFrame(data=(num_actv_bc_na.issue_y.value_counts(), num_actv_bc_zeros.issue_y.value_counts()),index=['num_actv_bc_tl_na','num_actv_bc_tl_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_actv_bc_tl_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_actv_bc_tl_zeros,NaN,NaN,NaN,NaN,NaN,385.0,1575.0,3230.0,5730.0,4816.0,3577.0,1738.0


In [0]:
# num_actv_rev_tl
num_actv_rev_tl_na=training.loc[training.num_actv_bc_tl.isnull(),['issue_y','id']]
num_actv_rev_tlc_zeros=training.loc[training.num_actv_bc_tl==0,['issue_y','id']]
pd.DataFrame(data=(num_actv_rev_tl_na.issue_y.value_counts(), num_actv_rev_tlc_zeros.issue_y.value_counts()),index=['num_actv_rev_tl_na','num_actv_rev_tl_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_actv_rev_tl_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_actv_rev_tl_zeros,NaN,NaN,NaN,NaN,NaN,385.0,1575.0,3230.0,5730.0,4816.0,3577.0,1738.0


In [0]:
# num_bc_sats
num_bc_sats_na=training.loc[training.num_bc_sats.isnull(),['issue_y','id']]
num_bc_sats_zeros=training.loc[training.num_bc_sats==0,['issue_y','id']]
pd.DataFrame(data=(num_bc_sats_na.issue_y.value_counts(), num_bc_sats_zeros.issue_y.value_counts()),index=['num_bc_sats_na','num_bc_sats_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_bc_sats_na,479.0,1906.0,4226.0,10071.0,17379.0,12805.0,NaN,NaN,NaN,NaN,NaN,NaN
num_bc_sats_zeros,NaN,NaN,NaN,NaN,NaN,295.0,709.0,1760.0,2848.0,2259.0,1608.0,589.0


In [0]:
# num_il_tl 
num_il_tl_na=training.loc[training.num_il_tl.isnull(),['issue_y','id']]
num_il_tl_zeros=training.loc[training.num_il_tl==0,['issue_y','id']]
pd.DataFrame(data=(num_il_tl_na.issue_y.value_counts(), num_il_tl_zeros.issue_y.value_counts()),index=['num_il_tl_na','num_il_tl_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_il_tl_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_il_tl_zeros,NaN,NaN,NaN,NaN,NaN,941.0,3976.0,5503.0,9023.0,5487.0,3283.0,1239.0


In [0]:
# num_op_rev_tl
num_op_rev_tl_na=training.loc[training.num_op_rev_tl.isnull(),['issue_y','id']]
num_op_rev_tl_zeros=training.loc[training.num_op_rev_tl==0,['issue_y','id']]
pd.DataFrame(data=(num_op_rev_tl_na.issue_y.value_counts(), num_op_rev_tl_zeros.issue_y.value_counts()),index=['num_op_rev_tl_na','num_op_rev_tl_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_op_rev_tl_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_op_rev_tl_zeros,NaN,NaN,NaN,NaN,NaN,5.0,34.0,49.0,73.0,92.0,78.0,34.0


In [0]:
# num_rev_accts
# Since it's highly correlated to num_op_rev_tl, so will drop this column
print('Correlation: ',training.num_rev_accts.corr(training.num_op_rev_tl))
num_rev_accts_na=training.loc[training.num_rev_accts.isnull(),['issue_y','id']]
num_rev_accts_zeros=training.loc[training.num_rev_accts==0,['issue_y','id']]
pd.DataFrame(data=(num_rev_accts_na.issue_y.value_counts().sort_index(), num_rev_accts_zeros.issue_y.value_counts()),index=['num_rev_accts_na','num_rev_accts_zeros'])

Correlation:  0.7744382737753008


,2007,2008,2009,2010,2011,2012,2015
num_rev_accts_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,1.0
num_rev_accts_zeros,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# sec_app_num_rev_accts
sec_app_num_rev_accts_na=training.loc[(training.application_type=='Joint App') & (training.sec_app_num_rev_accts.isnull()),['issue_y','id']]
sec_app_num_rev_accts_zeros=training.loc[(training.application_type=='Joint App') & (training.sec_app_num_rev_accts==0),['issue_y','id']]
pd.DataFrame(data=(sec_app_num_rev_accts_na.issue_y.value_counts().sort_index(), sec_app_num_rev_accts_zeros.issue_y.value_counts()),index=['sec_app_num_rev_accts_na','sec_app_num_rev_accts_zeros'])

,2015,2016,2017,2018
sec_app_num_rev_accts_na,306.0,3917.0,1117.0,NaN
sec_app_num_rev_accts_zeros,NaN,NaN,38.0,28.0


In [0]:
# num_sats
num_sats_na=training.loc[training.num_sats.isnull(),['issue_y','id']]
num_sats_zeros=training.loc[training.num_sats==0,['issue_y','id']]
pd.DataFrame(data=(num_sats_na.issue_y.value_counts().sort_index(), num_sats_zeros.issue_y.value_counts()),index=['num_sats_na','num_sats_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_sats_na,479.0,1906.0,4226.0,10071.0,17379.0,12805.0,NaN,NaN,NaN,NaN,NaN,NaN
num_sats_zeros,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,1.0,1.0,3.0,3.0


In [0]:
# num_tl_op_past_12m 
num_tl_op_past_12m_na=training.loc[training.num_tl_op_past_12m.isnull(),['issue_y','id']]
num_tl_op_past_12m_zeros=training.loc[training.num_tl_op_past_12m==0,['issue_y','id']]
pd.DataFrame(data=(num_tl_op_past_12m_na.issue_y.value_counts().sort_index(), num_tl_op_past_12m_zeros.issue_y.value_counts()),index=['num_tl_op_past_12m_na','num_tl_op_past_12m_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_tl_op_past_12m_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_tl_op_past_12m_zeros,NaN,NaN,NaN,NaN,NaN,4046.0,22423.0,29758.0,49704.0,33443.0,20451.0,6038.0


In [0]:
#tot_coll_amt
tot_coll_amt_na=training.loc[training.tot_coll_amt.isnull(),['issue_y','id']]
tot_coll_amt_zeros=training.loc[training.tot_coll_amt==0,['issue_y','id']]
pd.DataFrame(data=(tot_coll_amt_na.issue_y.value_counts().sort_index(), tot_coll_amt_zeros.issue_y.value_counts()),index=['tot_coll_amt_na','tot_coll_amt_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
tot_coll_amt_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
tot_coll_amt_zeros,NaN,NaN,NaN,NaN,NaN,19098.0,97821.0,150448.0,250925.0,180993.0,105870.0,32518.0


In [0]:
# tot_cur_bal
tot_cur_bal_na=training.loc[training.tot_cur_bal.isnull(),['issue_y','id']]
tot_cur_bal_zeros=training.loc[training.tot_cur_bal==0,['issue_y','id']]
pd.DataFrame(data=(tot_cur_bal_na.issue_y.value_counts().sort_index(), tot_cur_bal_zeros.issue_y.value_counts()),index=['tot_cur_bal_na','tot_cur_bal_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
tot_cur_bal_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
tot_cur_bal_zeros,NaN,NaN,NaN,NaN,NaN,7.0,19.0,36.0,33.0,30.0,60.0,98.0


In [0]:
# total_bal_ex_mort
total_bal_ex_mort_na=training.loc[training.total_bal_ex_mort.isnull(),['issue_y','id']]
total_bal_ex_mort_zeros=training.loc[training.total_bal_ex_mort==0,['issue_y','id']]
pd.DataFrame(data=(total_bal_ex_mort_na.issue_y.value_counts().sort_index(), total_bal_ex_mort_zeros.issue_y.value_counts()),index=['total_bal_ex_mort_na','total_bal_ex_mort_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
total_bal_ex_mort_na,479.0,1906.0,4226.0,10071.0,17379.0,5989.0,NaN,NaN,NaN,NaN,NaN,NaN
total_bal_ex_mort_zeros,NaN,NaN,NaN,NaN,NaN,21.0,43.0,67.0,66.0,87.0,103.0,131.0


In [0]:
# bc_util (drop)
# Can't explain why the nulls after 2012
bc_util_na=training.loc[training.bc_util.isnull(),['issue_y','id']]
bc_util_zeros=training.loc[training.bc_util==0,['issue_y','id']]
pd.DataFrame(data=(bc_util_na.issue_y.value_counts().sort_index(), bc_util_zeros.issue_y.value_counts()),index=['bc_util_na','bc_util_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
bc_util_na,479.0,1906.0,4226.0,10071.0,17379.0,6444.0,845.0,1990.0,3096.0,2500.0,1748.0,661.0
bc_util_zeros,NaN,NaN,NaN,NaN,NaN,376.0,822.0,1507.0,3073.0,2685.0,2089.0,1216.0


In [0]:
# num_bc_tl
num_bc_tl_na=training.loc[training.num_bc_tl.isnull(),['issue_y','id']]
num_bc_tl_zeros=training.loc[training.num_bc_tl==0,['issue_y','id']]
pd.DataFrame(data=(num_bc_tl_na.issue_y.value_counts().sort_index(), num_bc_tl_zeros.issue_y.value_counts()),index=['num_bc_tl_na','num_bc_tl_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
num_bc_tl_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_bc_tl_zeros,NaN,NaN,NaN,NaN,NaN,28.0,83.0,313.0,637.0,563.0,440.0,155.0


In [0]:
# mo_sin_old_il_acct (drop)
# Can't explain why the nulls after 2012
mo_sin_old_il_acct_na=training.loc[training.mo_sin_old_il_acct.isnull(),['issue_y','id']]
mo_sin_old_il_acct_zeros=training.loc[training.mo_sin_old_il_acct==0,['issue_y','id']]
pd.DataFrame(data=(mo_sin_old_il_acct_na.issue_y.value_counts().sort_index(), mo_sin_old_il_acct_zeros.issue_y.value_counts()),index=['mo_sin_old_il_acct_na','mo_sin_old_il_acct_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
mo_sin_old_il_acct_na,479.0,1906.0,4226.0,10071.0,17379.0,23084.0,3956.0,5492.0,9006.0,5477.0,3274.0,1237.0
mo_sin_old_il_acct_zeros,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,5.0,NaN,1.0,NaN


In [0]:
# mo_sin_old_rev_tl_op (Drop)
# Can't explain the null values
mo_sin_old_rev_tl_op_na=training.loc[training.mo_sin_old_rev_tl_op.isnull(),['issue_y','id']]
mo_sin_old_rev_tl_op_zeros=training.loc[training.mo_sin_old_rev_tl_op==0,['issue_y','id']]
pd.DataFrame(data=(mo_sin_old_rev_tl_op_na.issue_y.value_counts().sort_index(), mo_sin_old_rev_tl_op_zeros.issue_y.value_counts()),index=['mo_sin_old_rev_tl_op_na','mo_sin_old_rev_tl_op_zeros'])

,2007,2008,2009,2010,2011,2012
mo_sin_old_rev_tl_op_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0
mo_sin_old_rev_tl_op_zeros,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_rev_tl_op_na=training.loc[training.mo_sin_rcnt_rev_tl_op.isnull(),['issue_y','id']]
mo_sin_rcnt_rev_tl_op_zeros=training.loc[training.mo_sin_rcnt_rev_tl_op==0,['issue_y','id']]
pd.DataFrame(data=(mo_sin_rcnt_rev_tl_op_na.issue_y.value_counts().sort_index(), mo_sin_rcnt_rev_tl_op_zeros.issue_y.value_counts()),index=['mo_sin_rcnt_rev_tl_op_na','mo_sin_rcnt_rev_tl_op_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
mo_sin_rcnt_rev_tl_op_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
mo_sin_rcnt_rev_tl_op_zeros,NaN,NaN,NaN,NaN,NaN,319.0,1577.0,2874.0,4671.0,3517.0,2196.0,620.0


## 1.8 percent_bc_gt_75
**Solution:** Set all null to 0

Definition: Percentage of all bankcard accounts > 75% of limit

In [0]:
percent_bc_gt_75_na=training.loc[training.percent_bc_gt_75.isnull(),['issue_y','id','num_bc_tl','tot_hi_cred_lim']]
percent_bc_gt_75_zeros=training.loc[training.percent_bc_gt_75==0,['issue_y','id','num_bc_tl','tot_hi_cred_lim']]
num_bc_tl_na=training.loc[training.num_bc_tl.isnull(),['issue_y','id','percent_bc_gt_75']]
num_bc_tl_zeros=training.loc[training.num_bc_tl==0,['issue_y','id','percent_bc_gt_75']]

In [0]:
pd.DataFrame(data=(percent_bc_gt_75_na.issue_y.value_counts().sort_index(), percent_bc_gt_75_zeros.issue_y.value_counts(),num_bc_tl_na.issue_y.value_counts(),num_bc_tl_zeros.issue_y.value_counts()),index=['percent_bc_gt_75_na','percent_bc_gt_75_zeros','num_bc_tl_na','num_bc_tl_zeros'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
percent_bc_gt_75_na,479.0,1906.0,4226.0,10071.0,17379.0,6421.0,800.0,1946.0,3108.0,2400.0,1680.0,630.0
percent_bc_gt_75_zeros,NaN,NaN,NaN,NaN,NaN,6004.0,15299.0,30977.0,66709.0,60299.0,41935.0,16480.0
num_bc_tl_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,NaN,NaN,NaN,NaN,NaN,NaN
num_bc_tl_zeros,NaN,NaN,NaN,NaN,NaN,28.0,83.0,313.0,637.0,563.0,440.0,155.0


In [0]:
num_bc_tl_na.loc[num_bc_tl_na.percent_bc_gt_75==0,['issue_y']].issue_y.value_counts()

2012    3010
Name: issue_y, dtype: int64

## 1.9 pct_tl_nvr_dlq
**Solution:** Set all nulls to 100 percent

All the 100 happens after 2012, and most Null happens after 2012; thus we can make an educated guess that Null and 100 percent are the same.

In [0]:
# pct_tl_nvr_dlq
pct_tl_nvr_dlq_na=training.loc[training.pct_tl_nvr_dlq.isnull(),['issue_y','id']]
pct_tl_nvr_dlq_ones=training.loc[training.pct_tl_nvr_dlq==100,['issue_y','id']]
pd.DataFrame(data=(pct_tl_nvr_dlq_na.issue_y.value_counts().sort_index(), pct_tl_nvr_dlq_ones.issue_y.value_counts()),index=['pct_tl_nvr_dlq_na','pct_tl_nvr_dlq_ones'])

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
pct_tl_nvr_dlq_na,479.0,1906.0,4226.0,10071.0,17379.0,22140.0,125.0,NaN,NaN,NaN,NaN,1.0
pct_tl_nvr_dlq_ones,NaN,NaN,NaN,NaN,NaN,10472.0,60683.0,85856.0,142100.0,104111.0,63036.0,20983.0


## 1.10 annual_inc
**Solution:** Set Null in annual_inc_all to 0

There are two loans that individual applications with Null annual income, and in this case, we can assume those applicants don't have income then.

In joint applications, there are 230 primary applicants don't have income, but the secondary applicants have. Therefore set null annual_inc to 0 and the aggregated column, annual_inc_all to 0 too.

In [0]:
training.loc[training.annual_inc.isnull(),:].application_type.value_counts()

Individual    2
Name: application_type, dtype: int64

In [0]:
training.loc[training.annual_inc==0,:].application_type.value_counts()

Joint App    230
Name: application_type, dtype: int64

In [0]:
training.loc[(training.annual_inc==0) & (training.annual_inc_joint==0),:].application_type.value_counts()

Series([], Name: application_type, dtype: int64)

## 1.11 inq_last_6mths

**Solution:** Set Null inq_last_6mths to 0

Given the small volumn of the null values, we can assume null is interchangerable to 0.

In [0]:
training.loc[training.inq_last_6mths.isnull(),:].issue_y.value_counts()

2007    23
2016     1
Name: issue_y, dtype: int64

In [0]:
training.loc[training.inq_last_6mths==0,:].issue_y.value_counts().sort_index()

2007       140
2008       651
2009      1663
2010      4564
2011      8711
2012     21011
2013     55679
2014     94215
2015    180051
2016    130726
2017     75923
2018     22697
Name: issue_y, dtype: int64

##1.12 total_acc
**Solution:** Set null to median of total_acc, which is 23

In [0]:
training.loc[training.total_acc.isnull(),['id']].isin(training.loc[training.inq_last_6mths.isnull(),['id']].id).T

,34589,38454,96530,130885,132814,216760,270152,348344,376881,418784,423963,430890,569837,570716,665139,671022,680082,813180,890443,922551,959491,1003119,1003933
id,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [0]:
training.total_acc.median()

23.0

##1.13 revol_util & sec_app_revol_util

**Sotlution:** Drop both

Can't explain null in the context

In [0]:
training.loc[training.revol_util.isna(),['revol_util','issue_y','revol_bal']].revol_bal.value_counts()

0         548
146843      1
27922       1
6424        1
301         1
         ... 
8924        1
79589       1
81648       1
22258       1
50691       1
Name: revol_bal, Length: 134, dtype: int64

In [0]:
training.loc[ (training.application_type=='Joint App') & (training.sec_app_revol_util.isna()),['revol_util','issue_y','revol_bal']].revol_bal.value_counts()

0        32
7833      4
3060      4
7063      3
7002      3
         ..
29896     1
17610     1
5324      1
27855     1
34757     1
Name: revol_bal, Length: 5098, dtype: int64

# 2 Replacing Null Values

Null values will be replaced in the transformed dataset, training_tofill. Once finish this section, the dataset is ready to be used on model building.

In [84]:
training_tofill.columns.values

array(['id', 'acc_open_past_24mths', 'addr_state', 'application_type',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'desc', 'dti',
       'dti_joint', 'home_ownership', 'inq_last_6mths', 'loan_amnt',
       'num_bc_tl', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mths_since_recent_bc', 'num_actv_bc_tl',
       'num_actv_rev_tl', 'num_bc_sats', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_op_past_12m',
       'percent_bc_gt_75', 'purpose', 'revol_util', 'sec_app_revol_util',
       'term', 'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim',
       'total_acc', 'total_bal_ex_mort', 'pct_tl_nvr_dlq', 'results',
       'issue_d', 'issue_y', 'issue_m', 'earliest_cr_line1',
       'sec_app_earliest_cr_line1', 'earliest_cr_line_min',
       'verification_status_all', 'if_worse_rating', 'annual_inc_all',
       'revol_bal_all', 'mort_acc_all', 'num_rev_accts_all',
       'open_acc_all', 'if_revol_delinq', 'if_cur_il', 'i

In [85]:
training_tofill.shape

(1045085, 58)

##2.1 Replacing Null with 0

**Variables:**

acc_open_past_24mths, avg_cur_bal, tot_cur_bal, delinq_amnt, bc_open_to_buy, mort_acc, sec_app_mort_acc, num_actv_bc_tl, num_actv_rev_tl, num_bc_sats, num_il_tl, num_op_rev_tl, num_rev_accts, sec_app_num_rev_accts, num_sats, num_tl_op_past_12m, tot_coll_amt, tot_cur_bal, total_bal_ex_mort, num_bc_tl, mo_sin_old_rev_tl_op, mo_sin_rcnt_rev_tl_op, percent_bc_gt_75

In [0]:
tocombine=training.loc[:,['id','mort_acc','sec_app_mort_acc','num_rev_accts','sec_app_num_rev_accts']].fillna(0)

In [0]:
tocombine['mort_acc_all']=tocombine.mort_acc+tocombine.sec_app_mort_acc
tocombine['num_rev_accts_all']=tocombine.num_rev_accts+tocombine.sec_app_num_rev_accts

In [110]:
tocombine.head()

,id,mort_acc,sec_app_mort_acc,num_rev_accts,sec_app_num_rev_accts,mort_acc_all,num_rev_accts_all
0,1294383,0.0,0.0,2.0,0.0,0.0,2.0
1,1876502,0.0,0.0,6.0,0.0,0.0,6.0
2,954710,0.0,0.0,15.0,0.0,0.0,15.0
3,809314,5.0,0.0,19.0,0.0,5.0,19.0
4,1069627,3.0,0.0,9.0,0.0,3.0,9.0


In [0]:
training_tofill.fillna({'id':0,'acc_open_past_24mths':0,'avg_cur_bal':0,'tot_cur_bal':0,
                        'bc_open_to_buy':0,'num_actv_bc_tl':0,'num_actv_rev_tl':0,'num_bc_sats':0,
                        'num_il_tl':0,'num_op_rev_tl':0,'num_sats':0,'num_tl_op_past_12m':0,'tot_coll_amt':0,
                        'tot_cur_bal':0,'total_bal_ex_mort':0,'num_bc_tl':0,'mo_sin_old_rev_tl_op':0,
                        'mo_sin_rcnt_rev_tl_op':0,'percent_bc_gt_75':0,'annual_inc_all':0,'inq_last_6mths':0, 'num_bc_tl':0},
                       inplace=True)

In [112]:
training_tofill.shape

(1045085, 58)

In [113]:
training_tofill.drop(columns=['mort_acc_all','num_rev_accts_all'],inplace=True)
training_tofill.shape

(1045085, 56)

In [114]:
training_tofill=training_tofill.merge(tocombine.loc[:,['id','mort_acc_all','num_rev_accts_all']],on='id',validate='1:1')
training_tofill.shape

(1045085, 58)

In [115]:
training_tofill.isna().sum()[training_tofill.isna().sum()>0]

bc_util                        51345
dti                              239
dti_joint                    1026576
mo_sin_old_il_acct             85587
mths_since_recent_bc           50030
num_rev_tl_bal_gt_0            56201
revol_util                       681
sec_app_revol_util           1032165
tot_hi_cred_lim                56201
total_acc                         23
pct_tl_nvr_dlq                 56327
earliest_cr_line1                 23
sec_app_earliest_cr_line1    1031916
earliest_cr_line_min            5363
open_acc_all                      23
dtype: int64

##2.2 Filling with Special Values
- mths_since_recent_bc: Set null to max(mths_since_recent_bc)+1=640
- total_acc: Set null to median, 23
- pct_tl_nvr_dlq: Set all nulls to 100 percent
- dti: All null value in dti having a joint application, so in this case, use dti_joint instead.
- dti_joint: All the null come from individual applicaiton. For joint applications, use the weighted average of dti and dti_joint. 

  - The final dti can be calculated as annul_inc/(annual_inc + annual_inc_joint) * dti + annul_inc_joint/(annual_inc + annual_inc_joint) * dti_joint

In [0]:
training_tofill.fillna({'mths_since_recent_bc':640, 
                        'total_acc':23,'pct_tl_nvr_dlq':100
                        },inplace=True)

In [117]:
training_tofill.isna().sum()[training_tofill.isna().sum()>0]

bc_util                        51345
dti                              239
dti_joint                    1026576
mo_sin_old_il_acct             85587
num_rev_tl_bal_gt_0            56201
revol_util                       681
sec_app_revol_util           1032165
tot_hi_cred_lim                56201
earliest_cr_line1                 23
sec_app_earliest_cr_line1    1031916
earliest_cr_line_min            5363
open_acc_all                      23
dtype: int64

In [0]:
dtis=training.loc[:,['id','dti','dti_joint','application_type','annual_inc','annual_inc_joint']]
dtis['dti_all']=np.nan

In [119]:
dtis.head()

,id,dti,dti_joint,application_type,annual_inc,annual_inc_joint,dti_all
0,1294383,20.42,NaN,Individual,75000.0,NaN,NaN
1,1876502,7.97,NaN,Individual,25000.0,NaN,NaN
2,954710,11.52,NaN,Individual,32000.0,NaN,NaN
3,809314,23.66,NaN,Individual,135000.0,NaN,NaN
4,1069627,10.22,NaN,Individual,35000.0,NaN,NaN


In [0]:
# In the case of dti is null, fill them with 0
dtis.fillna({'dti':0},inplace=True)

# To handle the individual application_type:
# Let all dti_all equal to dti for individual applicaiton_type
Individual=dtis.loc[dtis.application_type=='Individual',['id','dti']]
dtis.loc[dtis['id'].isin(Individual['id']),['dti_all']]=Individual.loc[:,['dti']].values

In [121]:
# To handle the the joint application type:
# Let dti_add equal to 
# annul_inc/(annual_inc + annual_inc_joint) * dti + annul_inc_joint/(annual_inc + annual_inc_joint) * dti_joint
Joint=dtis.loc[dtis.application_type=='Joint App',:]
Joint.loc[:,'dti_all']=np.nan
Joint.loc[:,'dti_all']=Joint.loc[:,'annual_inc']/(Joint.loc[:,'annual_inc']+Joint.loc[:,'annual_inc_joint'])*Joint.loc[:,'dti']+Joint.loc[:,'annual_inc_joint']/(Joint.loc[:,'annual_inc']+Joint.loc[:,'annual_inc_joint'])*Joint.loc[:,'dti_joint']
dtis.loc[dtis['id'].isin(Joint['id']),['dti_all']]=Joint.loc[:,['dti_all']].values

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [122]:
dtis.isnull().sum()[dtis.isna().sum()>0]

dti_joint           1026576
annual_inc                2
annual_inc_joint    1026576
dtype: int64

In [123]:
dtis.shape

(1045085, 7)

In [0]:
# Merge the dti_all from dtis to training_tofill
training_tofill=training_tofill.merge(dtis.loc[:,['id','dti_all']],on='id')

In [125]:
training_tofill.shape

(1045085, 59)

##2.3 Droping Variables
'bc_util','mo_sin_old_il_acct','revol_util','sec_app_revol_util','mo_sin_old_rev_tl_op','num_rev_tl_bal_gt_0','open_acc_all','tot_hi_cred_lim','earliest_cr_line_min','dti','dti_joint','earliest_cr_line1','sec_app_earliest_cr_line1','annual_inc_all','issue_d','issue_m','issue_y'

- To drop num_rev_tl_bal_gt_0 as it's highly correlated with num_actv_rev_tl
- To drop open_acc_all as it's highly correlated with num_sats
- To drop tot_hi_cred_lim as it's highly correlated with tot_cur_bal
- To drop earliest_cr_line_min for simplication
- To drop ealiest_cr_line1
- To drop sec_app_earliest_cr_line1
- To drop time related variables, issue_d, issue_m, issue_y
- To drop 16 variables in total and total 43 variables left

In [126]:
training_tofill.drop(columns=['bc_util','mo_sin_old_il_acct','mo_sin_old_rev_tl_op',
                              'num_rev_tl_bal_gt_0','open_acc_all','tot_hi_cred_lim',
                              'earliest_cr_line_min','revol_util','sec_app_revol_util',
                              'dti','dti_joint','earliest_cr_line1','sec_app_earliest_cr_line1',
                              'issue_d','issue_m','issue_y'],inplace=True)
training_tofill.shape

(1045085, 43)

In [127]:
# As we can see, we've replaced all null values with alternatives
training_tofill.isna().sum()[training_tofill.isna().sum()>0]

Series([], dtype: int64)